# Query

In [ ]:
from wanglabconsts import OUTCOME_DEATH, OUTCOME_EDEMA

In [ ]:
from wanglabquerying import main

In [ ]:
import time

import numpy as np

from cyclops.processors.column_names import (
    ADMIT_TIMESTAMP,
    DIAGNOSIS_CODE,
    DIAGNOSIS_TRAJECTORY,
    ENCOUNTER_ID,
    EVENT_NAME,
    EVENT_TIMESTAMP,
    EVENT_VALUE,
    EVENT_VALUE_UNIT,
    TIMESTEP,
    YEAR,
)
from cyclops.processors.constants import (
    BINARY,
    CATEGORICAL_INDICATOR,
    FEATURE_INDICATOR_ATTR,
    FEATURE_MAPPING_ATTR,
    FEATURE_TYPE_ATTR,
    FEATURE_TYPES,
    FEATURES,
    MEAN,
    MIN_MAX,
    MISSING_CATEGORY,
    NUMERIC,
    ORDINAL,
    STRING,
    TARGETS,
)
from cyclops.processors.feature.split import intersect_datasets
from cyclops.utils.file import load_dataframe, save_dataframe

In [ ]:
t = time.time()
cohort, events = main()
print(time.time() - t)
cohort

In [ ]:
cohort[OUTCOME_DEATH].sum() / len(cohort)

In [ ]:
cohort[OUTCOME_EDEMA].sum() / len(cohort)

In [ ]:
# Intersect over encounter IDs to get only those encounters common to both
# cohort, events = intersect_datasets([cohort, events], ENCOUNTER_ID)

In [ ]:
save_dataframe(cohort, ENCOUNTERS_FILE)

# Clean / Preprocess

In [ ]:
events[EVENT_NAME] = normalize_names(events[EVENT_NAME])
events[EVENT_VALUE] = normalize_values(events[EVENT_VALUE])

In [ ]:
# Create the target as a timeseries event
death_events = cohort[cohort[OUTCOME_DEATH] == True]
death_events = death_events[[ENCOUNTER_ID, DISCHARGE_TIMESTAMP]]

# Offset death time - i.e., should predict death time in advance
death_events[DISCHARGE_TIMESTAMP] = death_events[DISCHARGE_TIMESTAMP] - pd.DateOffset(
    hours=PREDICT_OFFSET_HRS
)

death_events = death_events.rename({DISCHARGE_TIMESTAMP: EVENT_TIMESTAMP}, axis=1)
death_events[EVENT_NAME] = OUTCOME_DEATH_TEMP
death_events[EVENT_VALUE] = 1
death_events.head(5)

In [ ]:
events = pd.concat([events, death_events])

In [ ]:
save_dataframe(events, join(QUERIED_DIR, "batch_0000.parquet"))